# Ayudantía 05 - Iterables, iteradores, builts-in, y más...

__Autores: Pablo Sanhueza (@pabloist), Magdalena Nario(@magdalenanario), Francisca Cares(@franciscares)__

## Iterables
 
Los iterables son cualquier **objeto sobre el que se pueda iterar**, implementan el método `__iter__()`. Se puede iterar sobre ellos tantas veces como uno quiera. Algunos que ya conocemos 
* sets
* listas
* diccionarios
* deques

**NOTA**: un iterable no necesariamente tiene que ser indexable.

In [5]:
numeros_pares = {2,4,6,8,10,12}
for numero in numeros_pares:
    print(numero)

2
4
6
8
10
12


**IMPORTANTE:** Un iterable debe tener el método `__iter__` implementado, y debe retornar siempre un iterador. Por su parte, un iterador es un objeto que tiene el método `__next__` implementado, es decir puedo hacer next(iterador) y esto retornará un valor.

### Forma básica de un iterable
Necesitamos
1. Una clase.
2. Una clase iterable.
3. Una clase iterador.


## Iteradores

Un **iterador** es un objeto que **itera sobre un iterable**, es el objeto retornado por el método `__iter__()`, además contiene el método `__next__()` que nos retorna el siguiente elemento del iterable.

<div style="text-align: center;">
    <img src="./imagenes/iterador.png" width="700" height="700" />
</div>

In [91]:
x = [1, 2, 3]
print(next(x)) 

TypeError: 'list' object is not an iterator

Podemos ver que las lista x no es un iterador, sino ya tendría integrado el método next, por lo tanto lo convertiremos en un iterador utilizando `iter(x)`, lo cual es lo mismo que realizar `x.__iter__()`

In [93]:
iter1 = iter(x)
print(type(iter1))

<class 'list_iterator'>


In [94]:
print(next(iter1))
print(next(iter1))
print(next(iter1))
print(next(iter1))

1
2
3


StopIteration: 

Cuando no quedan objetos por recorrer el iterador debe levantar una excepción de tipo StopIteration.

Para crear un iterador, este debe contener los métodos `__iter__` y `__next__`

In [1]:
class Cuadrado:
    def __init__(self, numero):
        self.numero = numero

    def __iter__(self):
        return self

    def __next__(self):
        valor = self.numero ** 2
        self.numero = valor
        return valor

In [2]:
cuadrado = Cuadrado(2)
for i in range(5):
    print(next(cuadrado))

4
16
256
65536
4294967296


Al mismo tiempo podemos crear iterables personalizados con su propia clase iteradora

In [14]:
class Impuestos:
    def __init__(self):
        self.cuentas = [1000, 2000, 4500, 1200]
        self.ponderador = 1.19
        
    def __iter__(self):  
        return ImpuestoIterador(self)

In [15]:
class ImpuestoIterador:
    def __init__(self, iterable):
        self.iterable = iterable

    def __iter__(self):
        return self

    def __next__(self):
        if len(self.iterable.cuentas) == 0:
            raise StopIteration("Llegamos al final")
        else:
            siguiente = self.iterable.cuentas.pop(0) * self.iterable.ponderador
            return siguiente
    

Los *for loops* llaman al método `__iter__()` de la estructura iterable, y para obtener el siguiente valor en la iteración utilizan el método `__next__()` del iterador.

In [18]:
impuestos = Impuestos()
for i in impuestos:
    print(i)
    

1190.0
2380.0
5355.0
1428.0


## Generadores
Los generadores son un caso especial de los iteradores. Los generadores nos permiten iterar sobre secuencias de datos sin la necesidad de almacenarlos en alguna estructura de datos, evitando el uso innecesario de memoria. 

Esto es muy útil cuando queremos realizar cálculos sobre secuencias de números que sólo nos sirven para ese cálculo en particular. La sintaxis para crear generadores es muy parecida a la comprensión de listas, sólo que en vez de paréntesis cuadrados `[` `]` usamos paréntesis normales `(` `)`:

In [128]:
from sys import getsizeof
g = (b for b in range(100))
print("Tamaño Generador en bytes:" + str(getsizeof(g)))
l = [b for b in range(100)]
print("Tamaño lista en bytes:" + str(getsizeof(l)))

Tamaño Generador en bytes:120
Tamaño lista en bytes:912


Como los generadores son un caso especial de los iteradores, estos implementan ```__iter__``` permitiendo iterar sobre la secuencia y ```__next__``` para ver el siguiente elemento.
Es importante recordar que una vez que terminamos de iterar sobre un generador, el generador desaparece.

In [131]:
g = (b for b in range(10))
contador = 0
for b in g:
    print(b)
    contador += 1
    if contador == 9:
        break

0
1
2
3
4
5
6
7
8


In [61]:
print(next(g)) 
print(next(g))

9


StopIteration: 

## Funciones Generadoras

Las funciones en Python también tienen la posibilidad de funcionar como generadores, utilizando "yield". El statement "yield" equivale a "return". Por un lado se encarga de retornar el valor, pero además se asegura que en la próxima llamada a la función, la ejecución parta desde el punto donde se dejó en la ejecución anterior. 

Al llamar a una función generadora se crea un objeto generador, sin embargo, esto no comienza a ejecutar la función. Sino que lo hace cuando se itera sobre ella.

Ejemplo:
Consideremos la siguiente serie alternante:
$$ \sum_{n=1}^{m} n(-1)^{n-1}$$

De forma mas simple, lo anterior es equivalente a: 1 - 2 + 3 - 4 + 5 - ...
Hagamos una función generadora usando lo anterior.

In [2]:
def alternante(m):
    n = 1
    suma = 0
    while n < m:
        suma += (n * (-1) ** (n - 1))
        n += 1
        yield suma

generador_alternante = alternante(10)# notar que aquí no se imprime nada, 
                                     #ya que solo retorna el generador
print(type(generador_alternante))# se imprime el tipo de objeto

<class 'generator'>


In [3]:
for i in generador_alternante:
    print(i)

1
-1
2
-2
3
-3
4
-4
5


In [4]:
generador_alternante2 = alternante(5)
print(next(generador_alternante2))
print(next(generador_alternante2))
print(next(generador_alternante2))

1
-1
2


## Enviar valores a funciones generadoras
Es posible interactuar con funciones generadoras y enviarle datos usando el método `send()`.

Este método envía datos a la expresión `yield` de la función, los cuales pueden ser usados para asignarlo a otra variable, por ejemplo: 

In [17]:
def Descuentos(precio): 
    print(f'Aplicando descuentos a {precio} pesos')
    while True:
        porcentaje = yield
        ponderador = porcentaje/100
        valor_final = precio * (1 - ponderador)
        print(f'Se le aplicó un {porcentaje}% de descuento y el precio quedó en {valor_final}')
        
pesos = Descuentos(15000) #creo el objeto, no se ejecuta aún la función
next(pesos)#avanzo al primer yield, aquí se imprime "Aplicando descuentos ...."

pesos.send(10)
pesos.send(50)
pesos.send(15)

Aplicando descuentos a 15000 pesos
Se le aplicó un 10% de descuento y el precio quedó en 13500.0
Se le aplicó un 50% de descuento y el precio quedó en 7500.0
Se le aplicó un 15% de descuento y el precio quedó en 12750.0


## Built-ins 
### ```__len__```,```__getitem__```, ```enumerate```, ```zip```

Los _built-in_ son funciones que ya vienen implementadas en python, las cuales se pueden aplicar sobre distintos tipos de objetos.

Un primer ejemplo es ```__getitem__```, el cual permite acceder a elementos de algun objeto mediante el método de indexación.
Hay estructuras, como el caso de las listas, que ya tienen este método integrado lo cual nos permite acceder a los distintos elementos de las estas.

In [73]:
lista = [1, "maria", "rojo", True]
print(f'El tercer elemento de la lista es {lista[2]}')

El tercer elemento de la lista es rojo


Veamos que sucede al intentar indexar una clase

In [19]:
class Cronometro:
    def __init__(self, inicio):
        self.inicio = inicio
        
MiCronometro = Cronometro(20)
MiCronometro[0]

TypeError: 'Cronometro' object is not subscriptable

Esto sucede, ya que las clases no tienen el método ```__getitem__``` ya integrado, sin embargo este se puede definir dentro de la estructura de las clases de la siguiente forma

In [139]:
class Cronometro:
    def __init__(self, inicio):
        self.inicio = inicio
    
    def __getitem__(self, index):
        return f'El cronómetro inició en el minuto {self.inicio} y ahora va en el minuto {self.inicio+index}'
        
MiCronometro = Cronometro(20)
MiCronometro[30]

'El cronómetro inició en el minuto 20 y ahora va en el minuto 50'

## ```__len__```
La función `len()` aplicada a un objeto en particular hace un llamado a `objeto.__len__()`. La función `__len__` retorna el largo de un objeto y viene implementada en varias clases de estructuras de datos _built-in_

In [59]:
lista = [1, 2, 3, 4, 5]
print(len(lista))

5


Al mismo tiempo se puede utilizar en estructuras en las que no está integrado, para obtener el largo dependiendo de las condiciones que nosotros establezcamos.

Por ejemplo a continuación, está la clase ```Contagios```, la cual recibe un diccionario donde las llaves son los nombres de los alumnos de un curso y los valores corresponden a un bool dependiendo de si está contagiado con covid-19 o no.

In [83]:
class Contagios:
    def __init__(self, diccionario_curso):
        self.diccionario_curso = diccionario_curso
    def __len__(self):
        contagiados = 0
        for i in self.diccionario_curso.values():
            if i:
                contagiados += 1
        return contagiados

contagios = Contagios({"María": True, "Jose": False, "Camila": False, "Pedro": True, "Rocío": True})
print(len(contagios))

3


## ```enumerate```
```enumerate``` retorna una tupla, en la que el primer elemento es el indice que va enumerando, y el segundo es el item original que está siendo numerado. 

El primer parametro que se le entrega a ```enumerate``` debe ser un iterable, mientras que el segundo es el valor con el que se empieza a enumerar, que por defecto es 0.

Un claro ejemplo es cuado queremos asociar numeros a los meses del año, yendo desde el mes 1 correspondiente a enero hasta el mes 12 correspondiente a diciembre

In [21]:
inicio = 1
meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
for indice, mes in enumerate(meses, inicio):
    print(f"El mes numero {indice} es {mes}")

El mes numero 1 es Enero
El mes numero 2 es Febrero
El mes numero 3 es Marzo
El mes numero 4 es Abril
El mes numero 5 es Mayo
El mes numero 6 es Junio
El mes numero 7 es Julio
El mes numero 8 es Agosto
El mes numero 9 es Septiembre
El mes numero 10 es Octubre
El mes numero 11 es Noviembre
El mes numero 12 es Diciembre


Esta información puede quedar guardada a través de diccionarios de la siguiente forma

In [22]:
inicio = 1
diccionario_meses = dict()
meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
for indice, mes in enumerate(meses, inicio):
    diccionario_meses[indice] = mes

De esta forma podemos acceder rapidamente a los nombres de los meses, de acuerdo a su numeración

In [23]:
print(f'El mes numero {5} corresponde a {diccionario_meses[5]}')

El mes numero 5 corresponde a Mayo


## ```zip```

```zip``` toma dos o más iterables y va juntando su contenido en orden, es decir junta los primeros elementos de cada iterable, despues los segundos y así sucesivamente los va retornado en tuplas.
Por ejemplo, en el caso de que queramos unir a mascotas con sus dueños, podemos realizar lo siguiente


In [24]:
personas = ["María", "Sofía", "Pablo", "Martin"]
mascotas = ["perro", "conejo", "gato", "pez"]

In [142]:
print(list(zip(personas, mascotas)))
for persona, mascota in zip(personas, mascotas):
    print(f'{persona} es dueño de {mascota} ')

[('María', 'perro'), ('Sofía', 'conejo'), ('Pablo', 'gato'), ('Martin', 'pez')]
María es dueño de perro 
Sofía es dueño de conejo 
Pablo es dueño de gato 
Martin es dueño de pez 


Hay que recordar que zip toma la cantidad de elementos del iterable más corto

In [127]:
personas = ["María", "Sofía", "Pablo", "Martin", "José"]
mascotas = ["perro", "conejo", "gato", "pez"]
for persona, mascota in zip(personas, mascotas):
    print(f'{persona} es dueño de {mascota} ')

María es dueño de perro 
Sofía es dueño de conejo 
Pablo es dueño de gato 
Martin es dueño de pez 


Esto se puede realizar con múltiples iterables, por ejemplo en el ejemplo de los meses podemos agregar el iterable asociado a la cantidad dias que hay por mes

In [31]:
dias = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
for numero, mes, cantidad in zip(range(1,13), meses, dias):
    print(f'El mes {numero} corresponde a {mes} y tiene {cantidad} dias')

El mes 1 corresponde a Enero y tiene 31 dias
El mes 2 corresponde a Febrero y tiene 28 dias
El mes 3 corresponde a Marzo y tiene 31 dias
El mes 4 corresponde a Abril y tiene 30 dias
El mes 5 corresponde a Mayo y tiene 31 dias
El mes 6 corresponde a Junio y tiene 30 dias
El mes 7 corresponde a Julio y tiene 31 dias
El mes 8 corresponde a Agosto y tiene 31 dias
El mes 9 corresponde a Septiembre y tiene 30 dias
El mes 10 corresponde a Octubre y tiene 31 dias
El mes 11 corresponde a Noviembre y tiene 30 dias
El mes 12 corresponde a Diciembre y tiene 31 dias


## Funciones anónimas: 
## Función lambda

Antes de definirlas, recordemos que python tiene funciones de primera clase, es decir funciones que son tratadas como variables. Las que hemos usado siempre, como por ejemplo:

In [6]:
def saltar(cantidad, altura):
    return f"salté {cantidad} veces, y llegué {altura} metros de altura"

print(saltar(8,10))

salté 8 veces, y llegué 10 metros de altura


In [7]:
def actividad(actividad, fecha):
    return f"El día {fecha}, {actividad}"

print(actividad(saltar(9,5), '23 de Junio'))

El día 23 de Junio, salté 9 veces, y llegué 5 metros de altura


Funciones lambda son de ***primera clase*** o ***anónimas***. Tienen la siguiente estructura ```<lambda> <parámetros> : <retorno>`

In [8]:
multiplicar_anonimo = lambda x, y: x*y

print(multiplicar_anonimo(2,3))

6


In [9]:
def multiplicar(x,y):
    return x*y

print(multiplicar(2,3))

6


In [10]:
print(type(multiplicar_anonimo))
print(multiplicar_anonimo)
print(type(multiplicar))
print(multiplicar)

<class 'function'>
<function <lambda> at 0x000001B923349EE8>
<class 'function'>
<function multiplicar at 0x000001B923349C18>


Estas funciones pueden ser utilizas solo donde fueron creadas y de forma anónima.

## map
Son de la forma

`<map>(<funcion>:<iterables), <parametros>` 
Reciben una **función** y al menos un **iterable**.  

In [11]:
numeros = [1,2,3,4,5,6]
exponentes = [1,9,8,7,6,5]

potencia = map(lambda x,y: x**y,numeros, exponentes)

Intentemos imprimir potencia

In [12]:
print(potencia)

Notamos que vemos el type del objeto potencia. Para poder ver los elementos dentro de potencia debemos aplicarle `list` al resultado del mapeo.

In [13]:
print(list(potencia))

[1, 512, 6561, 16384, 15625, 7776]


Para trabajar con **más** de un iterable la función *lambda* deberá recibir el **mismo** número de parámetros que iterables, mientras que map iterará hasta el iterable de **menor** largo.

## filter
`filter(función: iterable)`

Recibe una **función** que retorne un **boolean** y un **iterable**. Retorna un **generador** con todos los elementos del iterable que retornaron **True**.

In [14]:
class Persona:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

Maida = Persona('Maida',18)
Pablo = Persona('Pablo',23)
Fran = Persona('Fran',21)
Ian = Persona('Ian',28)
Matias = Persona('Matias',15)
Juan = Persona('Juan',8)
Sofia = Persona('Sofia',5)

personas = [Maida, Pablo, Fran, Ian, Matias, Juan, Sofia]

Apliquemos un filter a la lista de personas, que nos retorne a todas las `Personas` que son menores de edad. Luego, imprimamos el resultado

In [15]:
menores_edad = filter(lambda x: x.edad <18,personas)
print(menores_edad)

Notemos que ocurre lo mismo que en el caso del mapeo. Por lo que para visualizar los objetos del generado retornado por el `filter` debemos aplicar `list` al generador resultante de aplicar el filter.

In [16]:
lista_menores = list(menores_edad)

Ahora, imprimamos el resultado del `filter`

In [17]:
for persona in lista_menores:
    print(f'{persona.nombre} tiene {persona.edad} años')

Matias tiene 15 años
Juan tiene 8 años
Sofia tiene 5 años


## Reduce
`<reduce>(<funcion>,<iterable>)`

Recibe una función que recibe **dos parámetros** y **un iterable**. Retorna la aplicación de la función de forma acumulada sobre el iterable. Y retorna el **resultado acumulado**.

**NOTA**: reduce se debe importar del módulo ***functools***

In [18]:
from functools import reduce

numeros = [9,8,72,5,6,4]

division_acumulada = reduce(lambda x,y: x/y,numeros)

print(division_acumulada)

0.00013020833333333333


# Ayudantía 05 - Ejercicio DCCumentos, Por favor!

<div style="text-align: center;">
    <img src="./imagenes/logo.jpg" />
</div>

Eres un(a) agente de aduanas post pandemia del Progravirus 2.0 👾, lo que significa: MILES DE VIAJEROS!
Todo el mundo está viajando ✈ usando los ahorros de toda su vida y su vejez, por lo que tienes N pega. Como agente de aduanas estás siempre alerta 🚨 de personas que pudiesen colocar en peligro la seguridad de tu país, **DCColekia**, éstos son identificables por sus pasaporte falsos que poseen errores o datos ilógicos ❓. Dada la gran cantidad de personas que se encuentran viajando, debes priorizar a investigar 🔎 solo aquellos que cumplan con ciertas características a modo de realizarles una revisión más exhaustiva o incluso una interrogación! 👮‍♂️ Para ello debes utilizar todos tus conocimientos de **Map, Filter, Reduce, lambda e iteradores**.

### Parte 1

En esta primera parte deberás realizar una investigación exhaustiva de todos los pasajeros que pasan a través de tu ventanilla 🕵️‍♂️, para ello tienes las siguientes características que generan **sospecha** sobre el pasajero:

* Color de ojos fuera de los comunes ("Verde", "Café" y "Azul")
* Fechas de nacimiento que no coincidan con la edad actual del pasajero
* Fechas de nacimiento antes del 1900 o posteriores al año actual
* Estaturas superiores 250 cm. e inferiores a 0 cm.
* Generos fuera de los estándar ("Masculino", "Femenino", "No binario")

Dado que los pasaportes de algunos países no tienen los más altos estándares de calidad 😅, debes retornar una lista con los sospechosos que cumplan **al menos 2** de las características anteriores.

### Parte 2

Pefecto, ya tienes a los sospechos ✔. En esta segunda parte debes interrogar a cada uno de ellos y observar a quién acusan. Ten mucho cuidado ⚠ porque tratarán de engañarte acusándose entre ellos generando un **Deadlock**

<div style="text-align: center;">
    <img src="./imagenes/Deadlock.png" />
</div>

Es por esto que cuando identifiques el **Deadlock** debes indicar a todos los que participen en él. Ellos son 💣.

IMPORTANTE: **Debes modelar todo el proceso con iteradores**

## Archivos

Para desarrollar el ejercicio cuentas con dos archivos, el primero **Personas.csv** comprende la información de los pasajeros para la parte 1 y tiene el siguiente formato:

Ej:

    Elsa patito; 1997; Santiago, 170, 23, Femenino, Verde
    Aquiles brinco, 2012, Shanghai, 176, 12, Masculino, Verde
    Xue Hua Piao Piao, 1999, Nueva York, 180, 25, No binario, Azul
    .
    .
    .

El segundo archivo corresponde a **Interrogaciones.csv**, el cual contiene información de un pasajero en particular y a **quién** acusaría en caso de ser interrogado, siguiendo el siguiente formato:

Ej:

    Elsa patito; Zoila Cerda
    Paula Daza, Elsa Patito
    Xue Hua Piao Piao, Elvis Tek
    .
    .
    .